# Import libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

In [ ]:
from tensorflow.keras.optimizers import RMSprop, SGD, Adam,Adamax
from tensorflow.keras import regularizers
from tensorflow.python.framework import ops
from tensorflow.keras.losses import Loss
from tensorflow.python.keras.utils import losses_utils
from tensorflow.python.keras.utils import tf_utils
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.python.ops import gen_math_ops
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, ZeroPadding2D,Dropout, Dense, Input, concatenate,GlobalAveragePooling2D
from tensorflow.keras.layers import  AveragePooling2D, Flatten,ZeroPadding2D,AveragePooling1D,Activation,BatchNormalization
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.optimizers import RMSprop, SGD, Adam,Adadelta,Adamax
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau,LearningRateScheduler
from keras.models import load_model
import math

# DATA *GENERATOR*

In [ ]:
#https://github.com/sayakpaul/ViT-jax2tf/blob/main/fine_tune.ipynb
BATCH_SIZE = 32
AUTO = tf.data.AUTOTUNE
num_classes = 101

def make_dataset(dataset: tf.data.Dataset, train: bool, image_size: int = 224):
    def preprocess(image, label):
        # For training, do a random crop and horizontal flip.
        if train:
            channels = image.shape[-1]
            begin, size, _ = tf.image.sample_distorted_bounding_box(
                tf.shape(image),
                tf.zeros([0, 0, 4], tf.float32),
                area_range=(0.05, 1.0),
                min_object_covered=0,
                use_image_if_no_bounding_boxes=True,
            )
            image = tf.slice(image, begin, size)

            image.set_shape([None, None, channels])
            image = tf.image.resize(image, [image_size, image_size])
            if tf.random.uniform(shape=[]) > 0.5:
                image = tf.image.flip_left_right(image)

        else:
            image = tf.image.resize(image, [image_size, image_size])

        image = (image - 127.5) / 127.5
        return image, label

    if train:
        dataset = dataset.shuffle(BATCH_SIZE * 10)
    return dataset.map(preprocess, AUTO).batch(BATCH_SIZE).prefetch(AUTO)

In [ ]:
#https://github.com/KlrShaK/Oxford_flowers102-using-Tensorflow/blob/master/Classifier/oxford_flower102.py
dataset_name = 'food101'
train_dataset, val_dataset = tfds.load(dataset_name, split=['train', 'validation'], as_supervised=True)
num_train = tf.data.experimental.cardinality(train_dataset)
num_val = tf.data.experimental.cardinality(val_dataset)
print(f"Number of training examples: {num_train}")
print(f"Number of validation examples: {num_val}")
train_dataset = make_dataset(train_dataset, True)
val_dataset = make_dataset(val_dataset, False)

Number of training examples: 75750
Number of validation examples: 25250


In [ ]:
b1_vit="https://tfhub.dev/sayakpaul/vit_b8_classification/1"
feature_extractor_model = b1_vit
feature_extractor_layer = hub.KerasLayer(feature_extractor_model,input_shape=(224,224 ,3),trainable=False)


# Model


In [ ]:
l2=tf.keras.regularizers.l2(1e-4)
l1l2=regularizers.l1_l2(l1=1e-5, l2=1e-4)
l1=tf.keras.regularizers.l1(1e-5)
kernel_init = tf.keras.initializers.he_normal()
l2=regularizers.l2(0.01)
bias_init = tf.keras.initializers.Constant(value=0.1)

In [ ]:
def inception_module1(x,
                     filters_3x3,
                     filters_3x3_1,
                     filters_3x3_2,
                     filters_5x5,
                     filters_5x5_1,
                     filters_5x5_2,
                     filters_pool_proj,
                     name='inc1'):

    conv_3x3 = Conv2D(filters_3x3, (1, 1), padding='same', activation='elu', kernel_initializer="he_normal",trainable=True)(x)
    conv_3x3_1 = Conv2D(filters_3x3_1, (3, 3), padding='same', activation='elu', kernel_initializer="he_normal",trainable=True)(conv_3x3)
    conv_3x3_2 = Conv2D(filters_3x3_2, (3, 3), padding='same', activation='elu', kernel_initializer="he_normal",kernel_regularizer=l2,trainable=True)(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5, (1, 1), padding='same', activation='elu', kernel_initializer=kernel_init,trainable=True)(x)
    conv_5x5_1= Conv2D(filters_5x5_1, (5, 5), padding='same', activation='elu', kernel_initializer="he_normal",trainable=True)(conv_5x5)
    conv_5x5_2 = Conv2D(filters_5x5_2, (5, 5), padding='same', activation='elu', kernel_initializer="he_normal",kernel_regularizer=l2,trainable=True)(conv_5x5)

    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='elu', kernel_initializer="he_normal")(pool_proj)

    output = concatenate([conv_3x3_1, conv_3x3_2, conv_5x5_1,conv_5x5_2, pool_proj], axis=3, name='inc1')

    return output

In [ ]:
def inception_module2(x,
                     filters_3x3,
                     filters_3x3_1,
                     filters_3x3_2,
                     filters_5x5,
                     filters_5x5_1,
                     filters_5x5_2,
                     filters_pool_proj,
                     name='inc21'):

    conv_3x3 = Conv2D(filters_3x3, (1, 1), padding='same', activation='elu', kernel_initializer="he_normal",trainable=True)(x)
    conv_3x3_1 = Conv2D(filters_3x3_1, (3, 3), padding='same', activation='elu', kernel_initializer="he_normal",trainable=True)(conv_3x3)
    conv_3x3_2 = Conv2D(filters_3x3_2, (3, 3), padding='same', activation='elu', kernel_initializer="he_normal",kernel_regularizer=l2,trainable=True)(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5, (1, 1), padding='same', activation='elu', kernel_initializer=kernel_init,trainable=True)(x)
    conv_5x5_1= Conv2D(filters_5x5_1, (5, 5), padding='same', activation='elu', kernel_initializer="he_normal",trainable=True)(conv_5x5)
    conv_5x5_2 = Conv2D(filters_5x5_2, (5, 5), padding='same', activation='elu', kernel_initializer="he_normal",kernel_regularizer=l2,trainable=True)(conv_5x5)

    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='elu', kernel_initializer="he_normal")(pool_proj)

    output = concatenate([conv_3x3_1, conv_3x3_2, conv_5x5_1,conv_5x5_2, pool_proj], axis=3, name='inc21')

    return output

In [ ]:
model = Sequential()
input_layer = Input(shape=(224,224,3))

vit =   feature_extractor_layer(input_layer)
x9 = Dense(num_classes, activation='softmax',name='vit_')(vit)

x_01=Conv2D(32, (3, 3), padding = 'same', kernel_initializer="he_normal",name='conv2d_10_')(input_layer)
x2=Activation('elu')(x_01)
x3=BatchNormalization(name='batch_10')(x2)

x4=Conv2D(64, (1,1), padding = "same", kernel_initializer="he_normal",name='conv1d_1')(x3)

x5=Conv2D(64, (3,3), padding = "same", kernel_initializer="he_normal",name='conv2d_102')(x4)
x6=Activation('elu')(x5)
x7=BatchNormalization()(x6)
x8=Dropout(0.5)(x7)

x9s=Conv2D(64, (3,3), padding = "same", kernel_initializer="he_normal",name='conv2d_103')(x8)
x0=Activation('elu')(x9s)
xy=BatchNormalization()(x0)

x0=AveragePooling2D(pool_size=(2, 2))(xy)

y1=AveragePooling2D(pool_size=(4, 4))(xy)
y1=Dropout(0.2)(y1)

x1=x0
##############################################
i0=Conv2D(64, (1,1), padding = "same", kernel_initializer="he_normal")(x1)

i1 = inception_module1(i0,
                     filters_3x3=128,
                     filters_3x3_1=192,
                     filters_3x3_2=192,
                     filters_5x5=64,
                     filters_5x5_1=96,
                     filters_5x5_2=96,
                     filters_pool_proj=64,
                     name='inc1')

i2 = inception_module2(i1,
                     filters_3x3=128,
                     filters_3x3_1=192,
                     filters_3x3_2=192,
                     filters_5x5=64,
                     filters_5x5_1=96,
                     filters_5x5_2=96,
                     filters_pool_proj=64,
                     name='inc2')

i4 = AveragePooling2D(pool_size=(6,6), padding='same', strides=None, name='max_pool_inception_01')(i1)
##############################################
x1z=Conv2D(128, (3, 3), padding = 'same', kernel_initializer="he_normal",name='conv2d_201')(x1)
x1x=Activation('elu')(x1z)
x1c=BatchNormalization(name='batch_12')(x1x)

x1v=Conv2D(128, (1,1), padding = "same", kernel_initializer="he_normal",name='conv1d_2')(x1c)

x13=ZeroPadding2D((1, 1))(x1v)
x12=Conv2D(128, (3, 3), padding = 'same', kernel_initializer="he_normal",name='conv2d_202')(x13)
x18=Activation('elu')(x12)
x19=BatchNormalization(name='batch_128')(x18)
x126=Conv2D(128, (3, 3), padding = 'valid', kernel_initializer="he_normal",name='conv203')(x19)
x1e=Activation('elu')(x126)
x23r=BatchNormalization()(x1e)
x23rr=ZeroPadding2D((2,2))(x23r)
x23t=Dropout(0.2)   (x23rr)
x23= MaxPooling2D(pool_size=(6,6))(x23t)
x21= AveragePooling2D(pool_size=(2, 2))(x23r)

x2t=concatenate([y1,x21])

x3r=Conv2D(256, (3, 3), padding = 'same', kernel_initializer="he_normal",name='conv2d_3_1')(x2t)
x3e=Activation('elu')(x3r)
x35w=BatchNormalization()(x3e)
x34q=Conv2D(256, (1,1), padding = "same", kernel_initializer="he_normal",name='conv1d_301')(x35w)
x36j=Conv2D(256, (3, 3), padding = 'same', kernel_initializer="he_normal",name='conv2d_302')(x34q)
x36h=Activation('elu')(x36j)
x3166g=BatchNormalization()(x36h)
x3166f=Dropout(0.2)(x3166g)
x316=ZeroPadding2D((1,1))(x3166f)
x33= MaxPooling2D(pool_size=(3,3))(x316)
x3_1=AveragePooling2D(pool_size=(2, 2)) (x3166f)
x41=ZeroPadding2D((1, 1))(x3_1)
x42=Conv2D(512, (3, 3), padding = 'same', kernel_initializer="he_normal",name='conv2d_401_')(x41)
x43=Activation('elu')(x42)
x44=BatchNormalization(name='batch521')(x43)
x65=Conv2D(512, (1,1), padding = "same", kernel_initializer="he_normal",name='conv1d_4_')(x44)
x66=Conv2D(512, (3, 3), padding = 'valid', kernel_initializer="he_normal",name='conv2d_402')(x65)
x67=Activation('elu')(x66)
x68=BatchNormalization()(x67)
x7 =AveragePooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x68)
dc_1_0 = Conv2DTranspose(512, (6,6), activation='elu',strides=1,
      padding = 'valid',kernel_initializer='he_normal', name='deconv_01_')(x7)
dc_1_0=concatenate([i4,dc_1_0,x23,x33])
dc_1 = UpSampling2D((2,2))(dc_1_0)
x7_0 = Flatten()(dc_1_0)
x7_1 = Dense(256, activation='elu',name='dense1')(x7_0)
x7_22 =BatchNormalization()(x7_1)
x7_2 = Dropout(0.1)(x7_22)
x7_4 = Dense(128, activation='elu',name='dense2')(x7_2)
x7_51 =BatchNormalization()(x7_4)
x7_5 = Dropout(0.1)(x7_51)
x8w = Dense(num_classes, activation='softmax',name='dense')(x7_5)
avr= tf.keras.layers.Average(name='average')([x8w,x9])
#########################
model = Model(input_layer, [x8w,x9,avr], name='Combination_Model_01')

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


# Training

In [ ]:
# Create tensorboard callback (functionized because need to create a new one for each model)
import datetime
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

#############################

  # Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 3 epochs
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # watch the val loss metric
                                                  patience=10) # if val loss decreases for 3 epochs in a row, stop training
##############################
# Creating learning rate reduction callback
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                 factor=0.2, # multiply the learning rate by 0.2 (reduce by 5x)
                                                 patience=2,
                                                 verbose=1, # print out when learning rate goes down
                                                 min_lr=1e-7)
###############################
def decay(epochs, steps=100):
    initial_lrate = 0.0001
    drop = 0.96
    epochs_drop = 8
    lrate = initial_lrate * math.pow(drop, math.floor((1+(2*epochs)/epochs_drop)))
    return lrate
################################
checkpoint = tf.keras.callbacks.ModelCheckpoint("/flower_1.h5",
                             #save the best model while all epovhs of training
                             monitor="val_loss",
                             save_best_only=True, # only save the best weights
                             save_weights_only=True, # only save model weights (not whole model)
                             mode="min",
                             verbose=1)

#################################
lr_sc = LearningRateScheduler(decay,verbose=1)
mycallback = [early_stopping,lr_sc,reduce_lr,checkpoint,
              create_tensorboard_callback(dir_name="tensorflow_hub",
              experiment_name="ENSEMBLE MODEL CIFAR10_Exp2")]

In [ ]:
model = tf.keras.models.load_model('/ensemble_food101_model_1')

In [ ]:
nb_train_samples = 75750
nb_validation_samples = 25250

lossWeights = {'dense3':0.25,'vit_':0.75}
loss5 = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer =Adamax(learning_rate=0.002, clipnorm=1.0)#optimizer =ad

model.compile( optimizer=optimizer, metrics=["accuracy"],loss={'dense3':loss5,'vit_':loss5},loss_weights=lossWeights)

history = model.fit(train_dataset
                    ,validation_data=val_dataset
                    ,epochs=100
                    ,callbacks=[mycallback])


Epoch 1: LearningRateScheduler setting learning rate to 9.6e-05.
Epoch 1/100
4735/4735 [==============================] - 2650s 557ms/step - loss: 2.9991 - dense3_loss: 4.3490 - vit__loss: 2.4869 - dense3_accuracy: 0.0623 - vit__accuracy: 0.4388 - average_accuracy: 0.4370 - val_loss: 1.7639 - val_dense3_loss: 4.0951 - val_vit__loss: 0.9400 - val_dense3_accuracy: 0.1048 - val_vit__accuracy: 0.7847 - val_average_accuracy: 0.7814 - lr: 9.6000e-05

Epoch 2: LearningRateScheduler setting learning rate to 9.6e-05.
Epoch 2/100
4735/4735 [==============================] - 2628s 555ms/step - loss: 2.1531 - dense3_loss: 4.0052 - vit__loss: 1.4985 - dense3_accuracy: 0.1088 - vit__accuracy: 0.6320 - average_accuracy: 0.6303 - val_loss: 1.4274 - val_dense3_loss: 3.6320 - val_vit__loss: 0.6637 - val_dense3_accuracy: 0.1683 - val_vit__accuracy: 0.8365 - val_average_accuracy: 0.8332 - lr: 9.6000e-05

Epoch 3: LearningRateScheduler setting learning rate to 9.6e-05.
Epoch 3/100
4735/4735 [=============